<a href="https://colab.research.google.com/github/EmmaVanPuyenbr/test/blob/main/voorbeeld_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Voorbeeld prof**

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
def splitData(dataX, dataY):
    trainX, testX, trainY, testY = train_test_split(dataX, dataY,test_size= 0.2,shuffle=True) # added shuffle=True 
    return trainX, testX, trainY, testY

In [ ]:
from tensorflow.keras.applications.vgg16 import VGG16
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import layers 
from tensorflow.keras import Model

class RandomClassificationModel:
    """
    Random classification model: 
        - generates random labels for the inputs based on the class distribution observed during training
        - assumes an input can have multiple labels
    """
    def fit(self, X, y):
        """
        Adjusts the class ratio variable to the one observed in y. 

        Parameters
        ----------
        X: list of arrays - n x (height x width x 3)
        y: list of arrays - n x (nb_classes)

        Returns
        -------
        self
        """
     
        trainX, testX, trainY, testY = splitData(X, y)

        # Add our data-augmentation parameters to ImageDataGenerator
        train_datagen = ImageDataGenerator(rescale = 1./255.,rotation_range = 40, width_shift_range = 0.2, height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True)

        # Note that the validation data should not be augmented!
        test_datagen = ImageDataGenerator( rescale = 1.0/255. )

        # Flow training images in batches of 20 using train_datagen generator
        train_generator = train_datagen.flow(trainX, batch_size = 20, class_mode = 'categorical', target_size = (224, 224))

        # Flow validation images in batches of 20 using test_datagen generator
        validation_generator = test_datagen.flow( testX,  batch_size = 20, class_mode = 'categorical', target_size = (224, 224))

        
        base_model = VGG16(input_shape = (224, 224, 3), # Shape of our images
                            include_top = False, # Leave out the last fully connected layer
                            weights = 'imagenet'
                            )
        
        for layer in base_model.layers:
          layer.trainable = False

        x = layers.Flatten()(base_model.output)
        x = layers.Dense(1024, activation='relu')(x)
        x = layers.Dropout(0.25)(x)
        x = layers.Dense(1, activation='sigmoid')(x)

        model = tf.keras.models.Model(base_model.input, x)

        model.compile(optimizer = tf.keras.optimizers.Adam(lr=0.0001), loss = 'binary_crossentropy',metrics = ['acc'])

        inc_history = model.fit_generator(train_generator, validation_data = validation_generator, steps_per_epoch = 100, epochs = 10)

        print(inc_history)
        return self
        
    def predict(self, X):
        """
        Predicts for each input a label.
        
        Parameters
        ----------
        X: list of arrays - n x (height x width x 3)
            
        Returns
        -------
        y_pred: list of arrays - n x (nb_classes)
        """
        np.random.seed(0)
        return [np.array([int(np.random.rand() < p) for p in self.distribution]) for _ in X]
    
    def __call__(self, X):
        return self.predict(X)
    
model = RandomClassificationModel()
model.fit(train_df["img"], train_df[labels])
test_df.loc[:, labels] = model.predict(test_df["img"])
test_df.head(1)

**Voorbeeld maarten**

In [ ]:
from sklearn.model_selection import train_test_split

from tensorflow.keras.applications.vgg16 import VGG16
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow.keras import layers 
from tensorflow.keras import Model
from sklearn.utils import shuffle

from keras.applications import MobileNetV2
from keras.models import Sequential, Model
from keras.layers import Activation, MaxPooling2D, BatchNormalization, Conv2D, Dense, GlobalAveragePooling2D, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

import cv2
import copy

In [ ]:
class MobileNetV2_TransferModel:
    """
    MobileNetV2_TransferModel: 
        - Transfer learning with a pre-trained model trained on ImageNet dataset
          The top layer of the network is removed, and new layers are added and trained to fit our data.
        - 'fit': trains top layer of the network. Steps before training the model: prepare data, split train-validate and preprocess
        - 'predict': predict on unseen test set.
        - assumes an input can have multiple labels
    """

    def fit(self, X, y, training=True):
        """
        Trains the toplayers of the pre-trained MobileNetV2 network.

        Parameters
        ----------
        X: list of arrays - n x (height x width x 3)
        y: list of arrays - n x (nb_classes)
        training: boolean indicating whether or not training must be performed. If False, the model was already trained 
        and stored and will be imported.

        Returns
        -------
        self
        """
        
        # prepare data
        X = X.apply(lambda x: cv2.resize(x,(224,224)))  # resize all images to same size
        X = np.stack(X,0)   # make numpy array with correct dimensions (N,X,Y,Z)
        y = np.array(y)     # dataframe to array

        # split train data into train-validate
        X_train, x_val, y_train, y_val = splitData(X, y)

        # preprocess 
        X_train = X_train/255.0  # normalize
        x_val = x_val/255.0

        X_train_mean = np.mean(X_train, axis=0) # mean substraction
        X_train = X_train - X_train_mean
        x_val = x_val - X_train_mean 
        self.X_train_mean = X_train_mean

        # checkpoint directory
        checkpoint_filepath = '/content/DATA/MyDrive/DATA/train/MobileNetV2_TransferModel.h5'

        if training:

          # MobileNetV2 basemodel (frozen)
          base_model = MobileNetV2(input_shape=(224,224,3), include_top=False, weights='imagenet')
          base_model.trainable = False

          # add layers to frozen MobileNetV2
          model = Sequential([
            base_model,
            GlobalAveragePooling2D(),
            Dense(20, activation='sigmoid')
          ])

          # Compile
          model.compile(optimizer='adam',
                        loss='binary_crossentropy',
                        metrics=['accuracy'])  # accuracy? or use f1-score?
          # data augmentation
          train_datagen = ImageDataGenerator(rotation_range=90, width_shift_range=0.2              
                                        ,height_shift_range=0.2, shear_range=0.2,zoom_range=0.2
                                        ,fill_mode="nearest", horizontal_flip=True)

          train_datagen.fit(X_train)

          # checkpoint
          save_best = ModelCheckpoint(filepath=checkpoint_filepath, monitor='val_accuracy', verbose=0, 
                                        save_best_only=True,save_weights_only=False, mode='max', save_freq="epoch")

          # training the classifier
          history = model.fit(train_datagen.flow(X_train, y_train, batch_size=32),
                                  validation_data=(x_val, y_val),                      
                                  epochs=50, verbose=1, workers=4, shuffle=True, callbacks=[save_best])
          self.model = model

        else:
          # load saved model
          self.model = load_model(checkpoint_filepath)

          return self
        
    def predict_(self, X):
        """
        Predicts for each input a label.
        
        Parameters
        ----------
        X: list of arrays - n x (height x width x 3)
            
        Returns
        -------
        y_pred: array - n x nb_classes
        """
        # prepare data
        print(X.shape)
        X = X.apply(lambda x: cv2.resize(x,(224,224)))  # resize all images to same size
        X = np.stack(X,0)   # make numpy array with correct dimensions (N,X,Y,Z)

        # preprocess
        X = X/255.0
        X = X - self.X_train_mean

        # predict
        y_pred = self.model.predict(X)
        return y_pred

    def __call__(self, X):
        return self.predict(X)
    
